# E-Commerce Clothing Classifer
## Image classification using convolutional neural networks in PyTorch

### Import modules

In [2]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Using data from FashionMNIST publically available clothing image datasets

In [3]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
])

In [4]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform = transforms)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms)

In [5]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
           )

In [6]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
           )

In [7]:
classes = train_data.classes
num_classes = len(train_data.classes)
print(f"classes: {classes}\nnum_classes: {num_classes}")

classes: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes: 10


In [8]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]
image_size

128

In [14]:
class MultiClassImageClassifier(nn.Module):
  
    def __init__(self):
        super(MultiClassImageClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 64 * 64, 10) 
        )

        
    def forward(self, x):
        x = self.classifier(x)
        return x

In [15]:
dataloader_train = DataLoader(train_data, shuffle=True, batch_size = 10,)

In [16]:
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_processed = 0
        for feature,label in dataloader_train:
            optimizer.zero_grad()
            outputs = net(feature)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(label)
        print(f"epoch:{epoch}, loss:{running_loss/num_processed}")
        
    train_loss = running_loss / len(dataloader_train)

In [24]:
net = MultiClassImageClassifier()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [25]:
train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

KeyboardInterrupt: 

In [19]:
dataloader_test = DataLoader(test_data,
                            shuffle = False,
                            batch_size =10,)

In [20]:
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()


In [21]:
print('Accuracy:', accuracy)
print('Precision (per class):\n', pd.DataFrame({"labels":classes,"precision":precision}))
print('Recall (per class):', recall)

Accuracy: 0.8715999722480774
Precision (per class):
         labels  precision
0  T-shirt/top   0.893035
1      Trouser   0.974026
2     Pullover   0.848283
3        Dress   0.778418
4         Coat   0.764023
5       Sandal   0.976884
6        Shirt   0.647003
7      Sneaker   0.959100
8          Bag   0.972782
9   Ankle boot   0.940661
Recall (per class): [0.7179999947547913, 0.9750000238418579, 0.765999972820282, 0.9449999928474426, 0.7900000214576721, 0.972000002861023, 0.6800000071525574, 0.9380000233650208, 0.9649999737739563, 0.9670000076293945]
